In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

In [2]:
from pathlib import Path 

In [3]:
workingdr = Path.cwd()

In [6]:
file_path = workingdr.joinpath('airquality-dataset', 'airquality-dataset', 'sample_dataset.csv')

In [7]:
assert file_path.exists()

In [10]:
df_airqo = pd.read_csv(file_path, index_col= 0) 
#Use the first column  as index

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [11]:
df_airqo.head()

,channel_id,pm2_5,pm10,s2_pm2_5,s2_pm10,Site,TimeStamp
0,912223,37.02,45.23,34.07,39.82,"Banda, Kampala",2019-11-27 13:24:45
1,912223,37.02,45.23,34.07,39.82,"Banda, Kampala",2019-11-27 13:24:45
2,912223,41.70,50.70,38.55,47.43,"Banda, Kampala",2019-11-27 13:26:13
3,912223,41.70,50.70,38.55,47.43,"Banda, Kampala",2019-11-27 13:26:13
4,912223,42.15,52.87,40.55,49.95,"Banda, Kampala",2019-11-27 13:30:53


In [12]:
df_airqo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23567841 entries, 0 to 25976646
Data columns (total 7 columns):
 #   Column      Dtype  
---  ------      -----  
 0   channel_id  int64  
 1   pm2_5       float64
 2   pm10        float64
 3   s2_pm2_5    float64
 4   s2_pm10     float64
 5   Site        object 
 6   TimeStamp   object 
dtypes: float64(4), int64(1), object(2)
memory usage: 1.4+ GB


In [13]:
df_airqo.describe()

,channel_id,pm2_5,pm10,s2_pm2_5,s2_pm10
count,2.356784e+07,2.356487e+07,2.356487e+07,2.356487e+07,2.356487e+07
mean,7.686704e+05,5.884781e+01,6.622911e+01,7.262280e+01,8.005443e+01
std,8.618918e+04,1.661815e+02,1.680471e+02,2.510575e+02,2.518958e+02
min,6.725280e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,6.897520e+05,2.657000e+01,2.915000e+01,2.690000e+01,2.962000e+01
50%,7.372760e+05,4.065000e+01,4.978000e+01,4.112000e+01,5.057000e+01
75%,8.701400e+05,5.927000e+01,7.085000e+01,6.028000e+01,7.175000e+01
max,9.304270e+05,4.299700e+03,1.345277e+04,4.379600e+03,8.438070e+03


In [15]:
df_airqo.isna().sum()

channel_id       0
pm2_5         2974
pm10          2974
s2_pm2_5      2973
s2_pm10       2974
Site             0
TimeStamp        0
dtype: int64

Few null values in airquality columns compared to the large data set of 23 million rows. The channel id, site and timestamp columns have no nulls. 

Exploring the data 

In [20]:
df_airqo.loc[df_airqo['pm10'].isnull() & ~df_airqo['s2_pm10'].isnull()]

,channel_id,pm2_5,pm10,s2_pm2_5,s2_pm10,Site,TimeStamp
10909716,782718,NaN,NaN,117.97,126.17,"Kasubi, Rubaga",2020-03-30 05:08:00


Only one column with null values for sensor one but with values for sensor 2 

In [22]:
df_airqo.loc[df_airqo['pm10'].isnull()]['Site'].unique()

array(['Butabika, Nakawa', 'Kasubi, Rubaga', 'Kisanga, Kasese'],
      dtype=object)

In [23]:
df_airqo['Site'].unique()

array(['Banda, Kampala', 'Bugana, Namayingo', 'Bugolobi, Nakawa',
       'Bukikali,Lwaso Mbale', 'Bukoto I, Kawempe',
       'Buloli North, Bududa', 'Bumurwa, Namisindwa', 'Busabala, Wakiso',
       'Busembatya, Iganga', 'Butabika, Nakawa',
       'Bweyogerere, Kira Municipality', 'Bwole Ward, Bugiri',
       'Central Division, Tororo', 'Civic Centre, Kampala Central',
       'Ggulu Ward, Mukono', 'Jinja Main Street, Jinja',
       'Kalerwe, Kawempe', 'Kasenge, Wakiso', 'Kasharara, Rubirizi',
       'Kasubi, Rubaga', 'Kisanga, Kasese', 'Kiwatule, Nakawa',
       'Kyaliwajala, Kira Municipality', 'Kyanja, Nakawa',
       'Kyebando, Kawempe', 'Lukuli, Makindye', 'Luwafu, Makindye',
       'Makindye, Lukuli Makindye', 'Masaba, Mbale Industrial division',
       'Mbalwa, Kira Municipality', 'Mbuya II, Nakawa',
       'Mutundwe, Makindye Ssabagabo', 'Naguru II, Nakawa',
       'Najera, Kira Municipality', 'Nansana east ward, Wakiso',
       'Nansana west ward, Wakiso', 'Nantabulirwa ward, M

Missing values for airquality for 6 sites 
Missing values can be filled with; 
1. Average of each site 
2. Average of each site within that day or within that hour 
3. Average per channel 
4. Drop them as a lazy way 

In [26]:
df_airqo['TimeStamp'] = pd.to_datetime(df_airqo['TimeStamp'])

In [33]:
#df_airqo.groupby(['Site', pd.Grouper(key="TimeStamp", freq="1H")]).agg(np.nanmean)
df_site_hourly_avg= df_airqo.groupby(['Site', df_airqo['TimeStamp'].dt.hour]).agg(np.nanmean)

In [29]:
df_airqo['TimeStampHour']= df_airqo['TimeStamp'].dt.hour

In [32]:
df_airqo.shape

(23567841, 8)

In [38]:
df_airqo.set_index(['Site', 'TimeStampHour'])['pm2_5'].fillna(df_site_hourly_avg['pm2_5'].to_dict())

Site                    TimeStampHour
Banda, Kampala          13               37.02
                        13               37.02
                        13               41.70
                        13               41.70
                        13               42.15
                                         ...  
Wobulenzi west, Luwero  13               81.33
                        13               57.38
                        13               43.15
                        13               46.45
                        13               42.53
Name: pm2_5, Length: 23567841, dtype: float64

Null pm2_5 values are filled with average hourly 

To do
1. Fill missing for pm10, sensor 2 pm2_5 and pm10 
2. Fix my github add readme about project 
3. Ignore data file 
4. 